In [9]:
#Load necessary packages
import tweepy
import folium
import geopy
import os
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim


from dotenv import load_dotenv
load_dotenv()

True

In [2]:
#set authentication variables
access_token_eve = os.environ.get('access_token')
access_secret_eve = os.environ.get('access_secret')
consumer_key_eve = os.environ.get('api_key')
consumer_secret_eve = os.environ.get('api_secret')

In [3]:
print(access_token_eve)

16810473-U802OXUMDjouQwt9Hq2N9odNrnUkfU00a1sRXgeVk


In [6]:
#load the Twitter API, create def for use
def get_twitter_api():
    # personal details
    consumer_key = consumer_key_eve
    consumer_secret = consumer_secret_eve
    access_token = access_token_eve
    access_token_secret = access_secret_eve
    
    # authentication of consumer key and secret
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    
    # authentication of access token and secret
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True)
    
    return api

#set up search for coordinates and/or location of the tweets
def get_twitter_location(search):
    api = get_twitter_api()
    count = 0
    for tweet in tweepy.Cursor(api.search_tweets, q=search).items(500):
        if hasattr(tweet, 'coordinates') and tweet.coordinates is not None:
            count += 1
            print("Coordinates", tweet.coordinates)
        if hasattr(tweet, 'location') and tweet.location is not None:
            count += 1
            print("Coordinates", tweet.location)
    print(count)
get_twitter_location("#Everton")

0


In [12]:
#set search for location of the Twitter user
def get_tweets(search):
    api = get_twitter_api()
    location_data = []
    for tweet in tweepy.Cursor(api.search_tweets, q=search).items(500):
        if hasattr(tweet, 'user') and hasattr(tweet.user, 'screen_name') and hasattr(tweet.user, 'location'):
            if tweet.user.location:
                location_data.append((tweet.user.screen_name, tweet.user.location))
    return location_data

In [14]:
print(location_data)

[('ThreeLionsThen', 'Bristol'), ('GangsterVeal', 'Liverpool'), ('fan_banter', 'United Kingdom'), ('TGoalpost', 'Malta'), ('bluekippercom', 'Liverpool, England'), ('Fanscapes1', 'Newbury, England'), ('CopperLebarante', 'Kumasi'), ('news9_sports', 'Noida'), ('LuvEvertonForum', 'Contact'), ('AndrewC_1999', 'Cork, Ireland'), ('News18Sports', 'India'), ('LuvEvertonForum', 'Contact'), ('matagonzalez86', 'San Miguel'), ('SaintDomingo78', 'Goodison '), ('Mimmi_Rirri', 'Lagos, Nigeria'), ('ShotongoalTH', 'Bangkok, Thailand'), ('CevascoCompleto', 'Reñaca Alto.'), ('PasiondeCerro', 'Viña del Mar'), ('INoticias_CL', 'Chile'), ('MaisOuiMaisOui_', 'Liverpool home of EVERTON FC'), ('richa_2as', 'عالم آخر '), ('LeePazzini', 'Liverpool'), ('Shanil7K', 'South East, England'), ('A_P_Ryan', 'Ireland'), ('LeePazzini', 'Liverpool'), ('nismoneil', 'Hereford, England'), ('gighozter', 'Indonesia'), ('andypeahead', 'bangor northern Ireland'), ('Jaygee_58', 'Edinburgh'), ('fan_banter', 'United Kingdom'), ('Roper

In [13]:
def put_markers(map, data):
    geo_locator = Nominatim(user_agent="EvertonTweeters")
    for (name, location) in data:
        if location:
            try:
                location = geo_locator.geocode(location)
            except GeocoderTimedOut:
                continue
            if location:
                folium.Marker([location.latitude, location.longitude], popup=name).add_to(map)

if __name__ == "__main__":
    map = folium.Map(location=[0, 0], zoom_start=2)
    location_data = get_tweets("#Everton")
    put_markers(map, location_data)
    map.save("index.html")

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Liverpool&format=json&limit=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000017FAE123D60>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=1)'))